## Imports

In [1]:
%load_ext cuml.accel

[2025-04-10 21:10:18.204] [CUML] [info] cuML: Installed accelerator for sklearn.
[2025-04-10 21:10:43.782] [CUML] [info] cuML: Installed accelerator for umap.
[2025-04-10 21:10:43.878] [CUML] [info] cuML: Installed accelerator for hdbscan.
[2025-04-10 21:10:43.878] [CUML] [info] cuML: Successfully initialized accelerator.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    f1_score, accuracy_score, precision_score,
    recall_score, confusion_matrix, roc_curve,
    auc, precision_recall_curve,
    average_precision_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost
import lightgbm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
import time
from sklearn.model_selection import StratifiedKFold

## Get the data

In [4]:
#Download the data
!curl -L -o /content/creditcardfraud.zip https://www.kaggle.com/api/v1/datasets/download/mlg-ulb/creditcardfraud

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 65.9M  100 65.9M    0     0  65.2M      0  0:00:01  0:00:01 --:--:-- 65.2M


In [5]:
#Extract the data
!unzip /content/creditcardfraud.zip -d /content/

Archive:  /content/creditcardfraud.zip
  inflating: /content/creditcard.csv  


## Initial Exploration

In [6]:
df = pd.read_csv('/content/creditcard.csv')

In [7]:
# Define a exibição de todas as colunas
pd.set_option('display.max_columns', None)

In [8]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [10]:
df.shape

(284807, 31)

In [11]:
# df.isna().sum()

In [12]:
# df.dtypes

In [13]:
# df.info()

In [14]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,2.239053e-15,1.673327e-15,-1.247012e-15,8.190001e-16,1.207294e-15,4.887456e-15,1.437716e-15,-3.772171e-16,9.564149e-16,1.039917e-15,6.406204e-16,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [15]:
count = df['Class'].value_counts()
percent = count / len(df) *100
print(count)
print()
print(percent)

Class
0    284315
1       492
Name: count, dtype: int64

Class
0    99.827251
1     0.172749
Name: count, dtype: float64


In [16]:
# # prompt: façar um plot com subblots para cada camada

# import matplotlib.pyplot as plt

# # Assuming 'df' is your DataFrame and it contains columns named 'V1' through 'V28'
# # Replace with your actual column names if different

# num_subplots = 28  # Number of V features (V1 to V28)
# fig, axes = plt.subplots(nrows=7, ncols=4, figsize=(20, 20))
# axes = axes.ravel()  # Flatten the axes array for easier indexing

# for i in range(1, num_subplots + 1):
#     ax = axes[i - 1]
#     column_name = f'V{i}'

#     # Create a histogram for each feature
#     ax.hist(df[df['Class'] == 0][column_name], alpha=0.5, label='Class 0', bins=30)
#     ax.hist(df[df['Class'] == 1][column_name], alpha=0.5, label='Class 1', bins=30)
#     ax.set_title(f'Distribution of {column_name}')
#     ax.set_xlabel(column_name)
#     ax.set_ylabel('Frequency')
#     ax.legend()

# # Adjust layout and show the plot
# plt.tight_layout()
# plt.show()


In [17]:
# for col in df.columns:
#   plt.figure(figsize=(10, 6))
#   plt.plot(df[col])
#   plt.title(col)
#   plt.show()

## Lets go



In [18]:
df_processed = df.copy()

scaler = StandardScaler()
df_processed['scaled_Amount'] = scaler.fit_transform(df_processed['Amount'].values.reshape(-1, 1))
df_processed['scaled_Time'] = scaler.fit_transform(df_processed['Time'].values.reshape(-1, 1))

df_processed.drop(['Time', 'Amount'], axis=1, inplace=True)

X = df_processed.drop(['Class'], axis=1)
y = df_processed['Class']

In [41]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [23]:
from cuml.svm import SVC as cumlSVC

In [30]:
N_SPLITS_CV = 5
RANDOM_STATE = 42

In [31]:
sampling_strategies = [1.0, 0.5, 0.3]
# sampling_strategies = [1.0, 0.5]

In [32]:
sampling_methods = {
    'None': None,
    'Random OverSampling': RandomOverSampler(random_state=42),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'Random UnserSampling': RandomUnderSampler(random_state=42),
    'Tomek Links': TomekLinks(),
    'ENN': EditedNearestNeighbours(),
    'SMOTEENN': SMOTEENN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42)
}

In [33]:
models = {
    'LogisticRegression': LogisticRegression(random_state=RANDOM_STATE, solver='saga'),
    'LogisticRegression_Balanced': LogisticRegression(random_state=RANDOM_STATE, solver='saga', class_weight='balanced'),
    'DecisionTree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'DecisionTree_Balanced': DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced'),
    'RandomForest': RandomForestClassifier(random_state=RANDOM_STATE),
    'RandomForest_Balanced': RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced_subsample'),
    'XGBoost': xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=RANDOM_STATE, verbosity=0),
    'SVM': cumlSVC(kernel='rbf', C=1.0, probability=True),
    'KNN': KNeighborsClassifier()
}

[2025-04-10 21:32:32.119] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization
[2025-04-10 21:32:32.119] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization
[2025-04-10 21:32:32.121] [CUML] [info] Unused keyword parameter: handle during CPU estimator initialization
[2025-04-10 21:32:32.121] [CUML] [info] Unused keyword parameter: verbose during CPU estimator initialization
[2025-04-10 21:32:32.121] [CUML] [info] Unused keyword parameter: output_type during CPU estimator initialization


In [34]:

#     'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1), # KNN pode ser lento
#     'DecisionTree': DecisionTreeClassifier(random_state=RANDOM_STATE),
#     'DecisionTree_Balanced': DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced'),
#     'RandomForest': RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100, n_jobs=-1),
#     'RandomForest_Balanced': RandomForestClassifier(
#         random_state=RANDOM_STATE, n_estimators=100, class_weight='balanced_subsample', n_jobs=-1),
#     'XGBoost': xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss', use_label_encoder=False, n_jobs=-1),
#     'LightGBM': lgb.LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1),
# }

In [35]:
# def evaluate_sampler(sampler, strategy, model, X_train, y_train, X_test, y_test):


#   if sampler is not None:
#     pipeline = Pipeline([('sampler', sampler), ('model', model)])
#     if strategy is not None:
#       sampler.sampling_strategy = strategy
#   if sampler is None:
#     pipeline = Pipeline([('model', model)])


#   pipeline.fit(X_train, y_train)
#   y_pred = pipeline.predict(X_test)
#   y_pred_prob = pipeline.predict_proba(X_test)[:, 1]

#   print(f'Resultado para {sampler} + {model}:')
#   print(classification_report(y_test, y_pred))

#   return roc_auc_score(y_test, y_pred_prob)

In [36]:
# for sampler_name, sampler in sampling_methods.items():
#   if sampler is None:
#     for model_name, model in models.items():
#       evaluate_sampler(sampler, None, model, X_train, y_train, X_test, y_test)
#   elif sampler_name == 'ENN' or sampler_name == 'Tomek Links':
#     for model_name, model in models.items():
#       evaluate_sampler(sampler, None, model, X_train, y_train, X_test, y_test)
#   else:
#     for strategy in sampling_strategies:
#       for model_name, model in models.items():
#         evaluate_sampler(sampler, strategy, model, X_train, y_train, X_test, y_test)

In [37]:
# def evaluate_sampler(samplers, models, strategies, X, y):
#   skf = StratifiedKFold(n_splits=N_SPLITS_CV, shuffle=True, random_state=RANDOM_STATE)
#   results = []

#   total_folds = skf.get_n_splits(X, y)
#   print(f'Starting validation with {total_folds} folds...\n')

#   for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
#     print(f'\nProcessing FOLD {fold}/{total_folds}')
#     start_fold_time = time.time()

#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

#     for sampler_name, sampler in samplers.items():
#       print(f'Sampler: {sampler_name}')
#       if sampler_name == 'None':
#         X_res, y_res = X_train, y_train
#         sampler_strategies = ['None']
#       else:
#         sampler_strategies = strategies

#       for strategy in sampler_strategies:
#         if sampler_name != 'None':
#           sampler.sampling_strategy = strategy
#           X_res, y_res = sampler.fit_resample(X_train, y_train)
#           print(f'Strategy: {strategy} | Resampling completed. Resample shape: {X_res.shape}')
#         else:
#           print(f'Strategy: {strategy} | No resampling')

#       for model_name, model in models.items():
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         y_prob = model.predict_proba(X_test)[:, 1]
#         print(classification_report(y_test, y_pred))


In [38]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, balanced_accuracy_score
)
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import StratifiedKFold



In [39]:
def evaluate_sampler(samplers, models, strategies, X, y):
    skf = StratifiedKFold(n_splits=N_SPLITS_CV, shuffle=True, random_state=RANDOM_STATE)
    results = []

    total_folds = skf.get_n_splits(X, y)
    print(f'Starting validation with {total_folds} folds...\n')

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
        print(f'Processing FOLD {fold}/{total_folds}')
        start_fold_time = time.time()

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        for sampler_name, sampler in samplers.items():
            print(f'\tSampler: {sampler_name}')
            if sampler_name == 'None':
                X_res, y_res = X_train, y_train
                sampler_strategies = ['None']
            else:
                sampler_strategies = strategies

            for strategy in sampler_strategies:
                if sampler_name != 'None':
                    sampler.sampling_strategy = strategy
                    X_res, y_res = sampler.fit_resample(X_train, y_train)
                    print(f'\t\tStrategy: {strategy} | Resampling completed. Resample shape: {X_res.shape}')
                else:
                    print(f'\t\tStrategy: {strategy} | No resampling applied.')

                for model_name, model in models.items():
                    print(f'\t\t\tTraining model: {model_name}')
                    start_model_time = time.time()

                    model.fit(X_res, y_res)
                    print(f'\t\t\t\tModel trained. Predicting...')

                    y_pred = model.predict(X_test)
                    y_prob = model.predict_proba(X_test)[:, 1]

                    model_duration = time.time() - start_model_time
                    print(f'\t\t\t\tComputing metrics...')
                    results.append({
                        'fold': fold,
                        'sampler': sampler_name,
                        'strategy': strategy,
                        'model': model_name,
                        'precision': precision_score(y_test, y_pred, zero_division=0),
                        'recall': recall_score(y_test, y_pred, zero_division=0),
                        'f1_score': f1_score(y_test, y_pred, zero_division=0),
                        'auc_roc': roc_auc_score(y_test, y_prob),
                        'gmean': geometric_mean_score(y_test, y_pred, average='binary'),
                        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
                        'confusion_matrix': confusion_matrix(y_test, y_pred),
                        'y_test': y_test,
                        'y_prob': y_prob,
                        'model_time': model_duration,
                        'fold_time': None
                    })

                    print(f'\t\t\t\tMetrics saved for model {model_name}')

        fold_duration = time.time() - start_fold_time
        print(f'Finished FOLD {fold} in {fold_duration:.2f}s\n')

        for r in results:
            if r['fold'] == fold and r['fold_time'] is None:
                r['fold_time'] = fold_duration

    print('\nCross validation completed.')

    results_df = pd.DataFrame(results)
    results_df.to_csv('results.csv', index=False)
    print(f'Results saved to results.csv')

    return results_df



In [40]:
evaluate_sampler(sampling_methods, models, sampling_strategies, X, y)

Starting validation with 5 folds...

Processing FOLD 1/5
	Sampler: None
		Strategy: None | No resampling applied.
			Training model: LogisticRegression
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model LogisticRegression
			Training model: LogisticRegression_Balanced
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model LogisticRegression_Balanced
			Training model: DecisionTree
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model DecisionTree
			Training model: DecisionTree_Balanced
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model DecisionTree_Balanced
			Training model: RandomForest
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model RandomForest
			Training model: RandomForest_Balanced
				Model trained. Predicting...
				Computing metrics...
				Metrics saved for model RandomForest_Balanced
			Training model: XGB

KeyboardInterrupt: 

In [103]:
# 📊 Função para gerar gráficos de AUC-ROC e AUC-PR:

# def plot_curvas(resultados_df, modelo, sampler='None', strategy='None'):
#     filtro = (
#         (resultados_df['modelo'] == modelo) &
#         (resultados_df['sampler'] == sampler) &
#         (resultados_df['strategy'] == strategy)
#     )

#     plt.figure(figsize=(8, 6))
#     for _, row in resultados_df[filtro].iterrows():
#         fpr, tpr, _ = roc_curve(row['y_test'], row['y_prob'])
#         plt.plot(fpr, tpr, label=f"Fold {row['fold']}")
#     plt.plot([0, 1], [0, 1], 'k--', label='Aleatório')
#     plt.title(f'Curva ROC - {modelo}')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     plt.figure(figsize=(8, 6))
#     for _, row in resultados_df[filtro].iterrows():
#         precision, recall, _ = precision_recall_curve(row['y_test'], row['y_prob'])
#         plt.plot(recall, precision, label=f"Fold {row['fold']}")
#     plt.title(f'Curva Precision-Recall - {modelo}')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.legend()
#     plt.grid(True)
#     plt.show()


In [102]:
# #💾 (Opcional) Salvar os resultados para reuso:
# # salvar como pickle
# resultados_df.to_pickle("resultados_validacao.pkl")

# # ou salvar como CSV (sem as listas, só métricas)
# resultados_df.drop(columns=['y_test', 'y_prob']).to_csv("resultados_validacao.csv", index=False)


In [100]:
results = {
    fold: {
        sampler_name: {
            strategy: {
                model_name: {'auc': 1, 'f1': 2}
                for model_name in models
            }
            for strategy in sampling_strategies
        }
        for sampler_name in sampling_methods
    }
    for fold in range(N_SPLITS_CV)
}

In [101]:
adsasd

{0: {'None': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model': {'auc': 1, 'f1': 2}, 'lr_model2': {'auc': 1, 'f1': 2}}},
  'Random OverSampling': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model': {'auc': 1, 'f1': 2}, 'lr_model2': {'auc': 1, 'f1': 2}}}},
 1: {'None': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model': {'auc': 1, 'f1': 2}, 'lr_model2': {'auc': 1, 'f1': 2}}},
  'Random OverSampling': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model': {'auc': 1, 'f1': 2}, 'lr_model2': {'auc': 1, 'f1': 2}}}},
 2: {'None': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model': {'auc': 1, 'f1': 2}, 'lr_model2': {'auc': 1, 'f1': 2}}},
  'Random OverSampling': {1.0: {'lr_model': {'auc': 1, 'f1': 2},
    'lr_model2': {'auc': 1, 'f1': 2}},
   0.5: {'lr_model':